In [1]:
%run stdPackages.ipynb
from lpEnergyModels.mBasic import * # Load everything from the mBasic module
import pickle

*Note: "`self`" is a placeholder that refers to the class MBasic or a specific instance of the class.*

# The MBasic class

The `MBasic` class builds primarily on three other classes: (1) The model structure class `symMaps.ModelShell`, (2) the linear programming class `symMaps.LPSys`, and (3) the database class `pyDbs.SimpleDB`. The relevant packages are documented here: 
* [GitHub: ChampionApe/symMaps](https://github.com/ChampionApe/symMaps)
* [GitHub: ChampionApe/pyDbs](https://github.com/ChampionApe/pyDbs).

The class `MBasic(ModelShell`) builds directly on the `symMaps.ModelShell` class. To each model instance, we set an attribute `self.sys` as the instance of the linear programming class `symMaps.LPSys` and a database instance `symMaps.SimpleDB` as the attribute `self.sys.db` (which can be accessed by calling `self.db` as well). 

The relevant Github pages include notebooks that more carefully describe the different classes. Here, we go through three steps: Data, model formulation, and solution.

## 1. Data

The model is a very simple power system model that is defined over four indices (parentheses indicate symbols used in plain math below).
* ```idxGen``` ($i$): Set of electricity generators.
* ```idxF``` ($f$): Set of fuel types that generators may use.
* ```idxEm``` ($em$): Set of emission types included in the model.
* ```idxCons``` ($ic$): Set of consumers in the power market.


The model requires the following data to run:
* ```pFuel(idxF)``` ($p_f$): Fuel price. Default unit: €/GJ. 
* ```uEm(idxF, idxEm)``` ($\phi_{f,em}$): Emission intensity. Default unit: Ton emission/GJ fuel input. 
* ```VOM(idxGen)``` ($c_i^{oth}$): variable operating and maintenance costs. Default unit: €/GJ output.
* ```uFuel(idxF, idxGen)``` ($\mu_{f,i}$): Fuelmix. Default unit: GJ input/GJ output.
* ```taxEm(idxEm)``` ($\tau_{em}$): Tax on emissions. Default unit: €/ton emission output.
* ```genCap(idxGen)``` ($q_i$): Generating capacity. Default unit: GJ/hour.
* ```mwp(idxCons)``` ($u_{ic}$): Marginal willingness to pay for consumers. Default unit: €/GJ. 
* ```load(idxCons)``` ($L_{ic}$): Total demand for consumers if the price does not exceed their marginal willingness to pay. Default unit: GJ.

The excel file `data/EX_MBasic.xlsx` in the data repo contains an example of the relevant data outlined above (except for values for the endogenous variables $E_i, D_{ic}$ that we are going to solve for). The file `data/EX_MBasic.pkl` in the data repo contains a pickled example of the excel data after reading it using the `pyDbs.ExcelSymbolLoadear` class.

This reads in the data and initializes a model instance that we can use to test out various methods:

In [2]:
testData = os.path.join(_d['data'],'EX_MBasic.xlsx')
data = pyDbs.ExcelSymbolLoader(testData)() # read data in a dict 
m = MBasic() # initialize model
[m.db.__setitem__(k, v) for k,v in data.items() if k!='__meta__']; # add data to model database

We can inspect the data from the database by calling:

In [3]:
m.db('idxGen') # inspect  index of generators included in the model

Index(['wind', 'solar', 'nuclear', 'biomass', 'natgas', 'coal'], dtype='object', name='idxGen')

Save pickle instance:

In [4]:
with open (os.path.join(_d['data'], 'EX_MBasic.pkl'), "wb") as file:
    pickle.dump(data, file)

## 2. Model formulation

### A. Formal setup

**Formal optimization problem:**

The model solves for vectors of electricity generation ($E_i$ = `generation`) and demand levels ($D_{ic}$ = `demand`) by solving:
$$\begin{align}
    &\max\mbox{ }\sum_{ic}u_{ic} D_{ic} - \sum_{i} c_i E_i \tag{1A} \\
    &\text{s.t. }\sum_{ic} D_{ic} = \sum_i E_i \tag{1B} \\
    & E_i\in[0,q_i], \quad \text{and} \quad D_{ic} \in[0, L_{ic}], \tag{1C}
\end{align}$$
and where the marginal costs of generation, $E_i$, are computed from:
$$\begin{align}
    c_i = c_i^{oth} + \sum_f \mu_{f,i} (p_f + \sum_{em} \phi_{f,em}\cdot\tau_{em}) \tag{2}
\end{align}$$

**Augmented form:**

The actual structure of the problem is handled by the `symMaps.LPSys`class, which is designed for Linear Programming (LP) problems. This means that we have to formulate the math in the following canonical form (we'll refer to this as an "augmented form"):
$$\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{3A}\\ 
    &A_{ub}\times x \leq b_{ub} \tag{3B}\\ 
    &A_{eq}\times x  = b_{eq} \tag{3C}\\ 
    &l\leq x\leq u, \tag{3D}
\end{align}$$
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

We generally refer to (3B)-(3C) as variational constraints, and (3D) as domain constraints.

### B. Add model structure -  `self.compile`

The method `self.compile` adds the model in equations (1)-(2) to the linear programming system `self.sys` consistent with the structure in equations (3). This is done in three steps:
1. Specify fundamentals: Provide variables and variational constraints in (1) and what sets/domains they are defined over.
2. An auxiliary calculations step: Provides a way of computing variables before the optimization, in our instance to calculate $c_i$ in (2) without adding this as a variational constraint in the LP model.
3. Add/specify parameters $c^T, A_{ub}, A_{eq}, b_{ub}, b_{eq}, l, u$.

####  *i)* Fundamentals - `self.compileMaps()`

The `self.compileMaps` starts by declaring domains of variables (in method `self.initArgs_v`), equality constraints (in method `self.initArgs_eq`), and inequality constraints (in method `self.initArgs_ub`). In this model, we have:
* Two variables: `generation` ($E_i$) defined over the index `idxGen` ($i$) and `demand` defined over the consumer index `idxCons`.
* One constraint: `equilibrium` which is a scalar constraint (index `None`).

Note that we include the auxiliary calculation for marginal costs $c_i$ outside the linear programming system; we revisit this below.

With this structure in place, the `self.compileMaps` method then applies `self.sys.compileMaps` which establishes the global indices for variables and constraints: 

In [5]:
m.compileMaps()
m.sys.v # look at the mapping from variable names to their relevant domains

{'generation': Index(['wind', 'solar', 'nuclear', 'biomass', 'natgas', 'coal'], dtype='object', name='idxGen'),
 'demand': Index(['residential', 'industrial'], dtype='object', name='idxCons')}

####  *ii)* Auxiliary calculations - `self.updateAux`

In the next step, we compute auxiliary variables. For this model, that means adding marginal costs `mc` to the `self.db` (following equation for $c_i$ above). 

In [6]:
m.updateAux()
m.db('mc') # inspect the marginal costs that we have added to the database

idxGen
biomass    14.00
coal       22.66
natgas     27.84
nuclear     2.15
solar       1.00
wind        1.00
dtype: float64

We implement this as a standard method in the class because we expect many scenarios likely involve recomputing this object. 

####  *iii)* Specify parameters for `LPSys` - `self.compileParams`

We specify the parameters for the `LPSys` method going through (1) adding coefficients on variables (`c`, `u`, `l`), (2) equality constraints (`A_eq`, `b_eq`), and (3) inequality constraints (`A_ub`, `b_ub`).

**Coefficients on variables:**

In our model, we have two variables to solve for `generation` and `demand`. We specify the coefficients (`c`, `l`, `u`) for each of the variables using methods with syntax `f'self.initArgsV_{k}'` with `k` representing the variable name (from `self.sys.v`). Specifically, we call the following:
```python
    [getattr(self, f'initArgsV_{k}')() for k in self.sys.v]; # specify c,l,u for all variables
```
Thus, if we extend the model with a new variable, we add it to `self.sys.v` (specify its domains) and add a method `self.initArgsV_{k}` that specifies the costs, lower bound, and upper bound. Given this, the `self.compile` method automatically includes this.

For the `generation` variable, for instance, we add the following:

```python
def initArgsV_generation(self):
	self.sys.lp['c'][('mc', 'generation')] = self.db('mc') 
	self.sys.lp['u'][('genCap', 'generation')] = self.db('genCap') 
```
We do not specify a lower bound, as the default is simply zero here.

**Equality constraints:**

Variational constraints (equality/inequality) are added in a similar manner as the coefficients on parameters:for equality constraints, we iterate through methods following a specific syntax `f'self.initArgsEq_{k}'`where `k` represents the specific constraint (from `self.sys.eq`). For inequality constraints, we simply replace `eq` with `ub`.

In this model, we have a single variational constraint in this model, namely the `equilibrium` one. Here we simply add:
```python
def initArgsEq_equilibrium(self):
	self.sys.lazyA('eq2Gen', series = 1,  v = 'generation', constr = 'equilibrium',attr='eq')
	self.sys.lazyA('eq2Dem', series = -1, v = 'demand', constr = 'equilibrium',attr='eq')
```
We rely on the `lazyA` method for adding the coefficients on `generation` and `demand`; this broadcasts the scalars (1 and -1, respectively) to the full domain of the constraint/variable. If the constraint requires a constant different from zero, we would have added it here to the `self.sys.lp['b_eq']` structure. Without adding any, the compiler defaults to zero. 

The `self.compileParams` sets up the structure and returns the `self.sys.out` that can be passed to the linear programming solver: 

In [7]:
m.compileParams()

{'c': array([  1.  ,   1.  ,   2.15,  14.  ,  27.84,  22.66, -30.  , -25.  ]),
 'A_ub': <COOrdinate sparse array of dtype 'float64'
 	with 0 stored elements and shape (0, 8)>,
 'b_ub': array([], dtype=float64),
 'A_eq': <COOrdinate sparse array of dtype 'int64'
 	with 8 stored elements and shape (1, 8)>,
 'b_eq': array([0.]),
 'bounds': array([[   0.,  720.],
        [   0.,  360.],
        [   0., 3600.],
        [   0.,  180.],
        [   0., 1440.],
        [   0., 2880.],
        [   0., 2000.],
        [   0., 7000.]])}

## 3. Post solution routine - `self.postSolve`

The steps above can be summarized by the following two calls:

In [8]:
m.compile()
sol = m.solve()

The `sol` is the solution object returned from `scipy.optimize.linprog`. Note that the `self.solve` includes an assertion that `sol['status'] == 0`, i.e. it raises an error if the optimization is not successful. For this model, we implement the following small `postSolve` routine:
```python
def postSolve(self, sol, **kwargs):
	solDict = super().postSolve(sol)
	solDict['surplus'] = -sol['fun']
	solDict['fuelCons'] = fuelConsumption(solDict['generation'], self.db('uFuel'))
	solDict['emissions'] = emissionsFuel(solDict['fuelCons'], self.db('uEm'))
	return solDict
```

Recall that the parent class (`ModelShell`) simply returns a dictionary with solution variables (`generation`, `demand`) and dual variables for variational constrains and domain constraints. On top of this, we add three variables: (1) A surplus, (2) fuel consumption, and (3) emissions. The `fuelConsumption` and `emissionsFuel` statements refer to locally defined functions. The solution from this statement:

In [9]:
solDict = m.postSolve(sol)
solDict.keys()

dict_keys(['generation', 'demand', 'λeq_equilibrium', 'λl_generation', 'λl_demand', 'λu_generation', 'λu_demand', 'surplus', 'fuelCons', 'emissions'])

If we want, we can add this directly to our database in `self.db` by calling the `self.postSolveToDB` instead; this draws directly on our customized `self.postSolve` routine and so, we automatically get the new variables with us here:

In [10]:
m.postSolveToDB(sol) # add solution to internal daatbase instead
m.db('generation').head() # look at solution for a bit

idxGen
wind        720.0
solar       360.0
nuclear    3600.0
biomass     180.0
natgas        0.0
Name: generation, dtype: float64